In [1]:
import pandas as pd

In [2]:
# That didn't work super well, let's try to parse the XML manually

import xml.etree.ElementTree as ET

tree = ET.parse("../../data/bioml/sra_metadata.xml")
root = tree.getroot()
root

<Element 'BioSampleSet' at 0x7f39f9a84db0>

In [3]:
# Find all the BioSample elements
biosamples = root.findall(".//BioSample")

# Create a dictionary to store the IDs for each sample
sample_ids = {}

# Iterate over each BioSample element
for biosample in biosamples:
    # Get the sample accession
    accession = biosample.get("accession")
    # print(accession)

    # Find all the Id elements within the BioSample
    ids = biosample.findall(".//Id")

    # Create a dictionary to store the IDs for the current sample
    sample_ids[accession] = {}

    # Iterate over each Id element and extract the ID values
    for id_elem in ids:
        id_value = id_elem.text
        if "db" in id_elem.attrib:
            db = id_elem.get("db")
        elif "db_label" in id_elem.attrib:
            db = id_elem.get("db_label")
        else:
            db = "unknown"
        sample_ids[accession][db] = id_value
    # break

# Print the extracted IDs for each sample
for accession, ids in sample_ids.items():
    print(f"Sample Accession: {accession}")
    for db, id_value in ids.items():
        print(f"  {db}: {id_value}")
    print()

Sample Accession: SAMN11950562
  BioSample: SAMN11950562
  Sample name: dl-0006_MG
  SRA: SRS4899508

Sample Accession: SAMN11950561
  BioSample: SAMN11950561
  Sample name: dl-0001_MG
  SRA: SRS4899507

Sample Accession: SAMN11950560
  BioSample: SAMN11950560
  Sample name: dk-0003_MG
  SRA: SRS4899944

Sample Accession: SAMN11950559
  BioSample: SAMN11950559
  Sample name: dk-0001_MG
  SRA: SRS4899890

Sample Accession: SAMN11950558
  BioSample: SAMN11950558
  Sample name: dj-0016_MG
  SRA: SRS4899891

Sample Accession: SAMN11950557
  BioSample: SAMN11950557
  Sample name: dj-0001_MG
  SRA: SRS4899896

Sample Accession: SAMN11950556
  BioSample: SAMN11950556
  Sample name: di-0009_MG
  SRA: SRS4899555

Sample Accession: SAMN11950555
  BioSample: SAMN11950555
  Sample name: di-0001_MG
  SRA: SRS4899894

Sample Accession: SAMN11950554
  BioSample: SAMN11950554
  Sample name: dh-0010_MG
  SRA: SRS4899895

Sample Accession: SAMN11950553
  BioSample: SAMN11950553
  Sample name: dh-0001_MG

In [4]:
# Convert to dataframe
df = pd.DataFrame(sample_ids).T
df

,BioSample,Sample name,SRA
SAMN11950562,SAMN11950562,dl-0006_MG,SRS4899508
SAMN11950561,SAMN11950561,dl-0001_MG,SRS4899507
SAMN11950560,SAMN11950560,dk-0003_MG,SRS4899944
SAMN11950559,SAMN11950559,dk-0001_MG,SRS4899890
SAMN11950558,SAMN11950558,dj-0016_MG,SRS4899891
...,...,...,...
SAMN11846034,SAMN11846034,am_0014_0091_c7,SRS4883616
SAMN11846033,SAMN11846033,am_0014_0091_b8,SRS4883609
SAMN11846032,SAMN11846032,am_0014_0091_b12,SRS4883610
SAMN11846031,SAMN11846031,am_0014_0091_a12,SRS4883611


In [5]:
from collections import Counter

all_suffixes = Counter([str(x).split(" ")[-1].split("_")[-1].split("-")[-1] for x in df["Sample name"]])
all_suffixes.most_common(10)

[('WGS', 3632),
 ('16S', 1168),
 ('MG', 563),
 ('BIS', 103),
 ('e2', 49),
 ('g5', 48),
 ('e3', 48),
 ('d2', 48),
 ('c2', 47),
 ('b3', 47)]

In [6]:
mapping = {}
for i, row in df.iterrows():
    if row["Sample name"].endswith("16S"):
        name = str(row["Sample name"])[:-4]
        info = {"16S SRA": row["SRA"], "16S Biosample": row["BioSample"]}
    elif row["Sample name"].endswith("WGS"):
        name = str(row["Sample name"])[:-4]
        info = {"WGS SRA": row["SRA"], "WGS Biosample": row["BioSample"]}
    else:
        continue  # Skip samples that don't have either

    if name not in mapping:
        mapping[name] = {}
    # mapping[name].update(info)
    for k, v in info.items():
        mapping[name][k] = v

# Convert to dataframe
df_mapping = pd.DataFrame(mapping).T
# df_mapping = df_mapping[df_mapping["16S SRA"].notnull() | df_mapping["WGS SRA"].notnull()]
df_mapping

,WGS SRA,WGS Biosample,16S SRA,16S Biosample
bj_0095_0068_d7,NaN,SAMN11946669,NaN,NaN
bj_0095_0068_c6,NaN,SAMN11946668,NaN,NaN
bj_0095_0068_g5,NaN,SAMN11946667,NaN,NaN
bj_0095_0068_b6,NaN,SAMN11946666,NaN,NaN
bj_0095_0068_a7,NaN,SAMN11946665,NaN,NaN
...,...,...,...,...
ac-0038,NaN,NaN,SRS4894852,SAMN11941247
ac-0002,NaN,NaN,SRS4894860,SAMN11941246
ab-0140,NaN,NaN,SRS4894858,SAMN11941245
aa-0163,NaN,NaN,SRS4894859,SAMN11941244


In [7]:
# Ok, we should take another approach. Here's the metadata from the paper
# !pip install openpyxl
import pandas as pd

metadata = pd.read_excel("../../data/bioml/41591_2019_559_MOESM3_ESM.xlsx", sheet_name="SupTable3", header=1)
ids = metadata[(metadata[["16S", "metagenomes"]] == "YES").all(axis=1)]["sample-id"]

/home/phil/mambaforge/envs/aptr/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
# Check how many of the IDs match via the mapping:
# name_MG --> WGS
# name_16S --> 16S

matching = 0
mg = 0
s16 = 0
total = 0
for id in ids:
    total += 1
    if df["Sample name"].str.startswith(id).any():
        matching += 1
        if (df["Sample name"] == id + "_MG").any():
            mg += 1
        if (df["Sample name"] == id + "_16S").any():
            s16 += 1

matching, total, mg, s16

# Ok great, it's a perfect match.

(541, 541, 541, 541)

In [9]:
# Well, let's build our dataframe

df_reindexed = df.set_index("Sample name")
sra_df = pd.DataFrame(columns=["WGS SRA", "WGS Biosample", "16S SRA", "16S Biosample"])
for id in ids:
    sra_df.loc[id] = [
        df_reindexed.loc[id + "_MG"]["SRA"],
        df_reindexed.loc[id + "_MG"]["BioSample"],
        df_reindexed.loc[id + "_16S"]["SRA"],
        df_reindexed.loc[id + "_16S"]["BioSample"],
    ]
sra_df

,WGS SRA,WGS Biosample,16S SRA,16S Biosample
aa-0154,SRS4899874,SAMN11950000,SRS4894856,SAMN11941243
aa-0163,SRS4899871,SAMN11950001,SRS4894859,SAMN11941244
ab-0140,SRS4899875,SAMN11950002,SRS4894858,SAMN11941245
ac-0002,SRS4899924,SAMN11950004,SRS4894860,SAMN11941246
ac-0038,SRS4899923,SAMN11950005,SRS4894852,SAMN11941247
...,...,...,...,...
dj-0016,SRS4899891,SAMN11950558,SRS4895434,SAMN11942406
dk-0001,SRS4899890,SAMN11950559,SRS4895431,SAMN11942407
dk-0003,SRS4899944,SAMN11950560,SRS4895432,SAMN11942408
dl-0001,SRS4899507,SAMN11950561,SRS4895429,SAMN11942409


In [10]:
# And while we're at it, let's go ahead and dump the 16S and WGS reads!

import os
from tqdm import tqdm

# Verify we have fastq-dump
print(os.system("fastq-dump --version"))

# Start with 16S
# for seqtype in ["16S", "WGS"]:
for seqtype in ["WGS"]:  # Re-run this later
    os.makedirs(f"/home/phil/DATA1/bioml/{seqtype}", exist_ok=True)
    os.makedirs(f"../../data/bioml/reads/{seqtype}", exist_ok=True)
    for id in tqdm(ids):
        # print(f"Dumping {seqtype} reads for {id}")
        sra = sra_df.loc[id][f"{seqtype} Biosample"]
        os.system(f"fastq-dump --split-files {sra} -O ~/DATA1/bioml/{seqtype}/ > /dev/null 2>&1")

        # Add symlinks: Data1/.../SRA <-- reads/.../id
        os.system(
            f"ln -s ~/DATA1/bioml/{seqtype}/{sra}_1.fastq ../../data/bioml/reads/{seqtype}/{id}_1.fastq > /dev/null 2>&1"
        )
        os.system(
            f"ln -s ~/DATA1/bioml/{seqtype}/{sra}_2.fastq ../../data/bioml/reads/{seqtype}/{id}_2.fastq > /dev/null 2>&1"
        )


fastq-dump : 3.1.1

0


  0%|          | 2/541 [16:24<72:40:33, 485.40s/it]